# Pytorch - Redes Neuronales

In [ ]:
import torch

In [ ]:
#Acceder a Drive
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import numpy as np

Mounted at /content/drive


In [ ]:
#data = np.loadtxt("/content/drive/MyDrive/Colab Notebooks/Datasets/trainmymnist.csv", delimiter=',',skiprows=1)
data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Datasets/dataset_alphanum.csv")
# print(data)
data = data.to_numpy()
X, Y = data[:, 1:], data[:, 0]
print(X.shape)
print(Y.shape)

(4657, 784)
(4657,)


In [ ]:
D_in, H, D_out = 784, 100, 27

model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out),
)

In [ ]:
outputs = model(torch.randn(64, 784))
outputs.shape

torch.Size([64, 27])

In [ ]:
model.to("cuda")

Sequential(
  (0): Linear(in_features=784, out_features=100, bias=True)
  (1): ReLU()
  (2): Linear(in_features=100, out_features=27, bias=True)
)

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
# importando libreria numpy
#import numpy as np

# normalización y split

X_train, X_test, y_train, y_test = X[:50000] / 255., X[50000:] / 255., Y[:50000].astype(int), Y[50000:].astype(int)

In [ ]:
# función de pérdida y derivada

def softmax(x):
    return torch.exp(x) / torch.exp(x).sum(axis=-1,keepdims=True)

def cross_entropy(output, target):
    logits = output[torch.arange(len(output)), target]
    loss = - logits + torch.log(torch.sum(torch.exp(output), axis=-1))
    loss = loss.mean()
    return loss

In [ ]:
print(X)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [ ]:
model.cuda()

Sequential(
  (0): Linear(in_features=784, out_features=100, bias=True)
  (1): ReLU()
  (2): Linear(in_features=100, out_features=27, bias=True)
)

In [ ]:
# convertimos datos a tensores y copiamos en gpu

X_t = torch.from_numpy(X_train).float().cuda()
Y_t = torch.from_numpy(y_train).long().cuda()

# bucle entrenamiento
epochs = 100
lr = 0.6
log_each = 10
l = []
for e in range(1, epochs + 1):

    # forward
    y_pred = model(X_t)

    # loss
    loss = cross_entropy(y_pred, Y_t)
    l.append(loss.item())

    # ponemos a cero los gradientes
    model.zero_grad()

    # Backprop (calculamos todos los gradientes automáticamente)
    loss.backward()

    # update de los pesos
    with torch.no_grad():
        for param in model.parameters():
            param -= lr * param.grad

    if not e % log_each:
        print(f"Epoch {e}/{epochs} Loss {np.mean(l):.5f}")

Epoch 10/100 Loss 3.02897
Epoch 20/100 Loss 2.58431
Epoch 30/100 Loss 2.33872
Epoch 40/100 Loss 2.14233
Epoch 50/100 Loss 2.00057
Epoch 60/100 Loss 1.89175
Epoch 70/100 Loss 1.79853
Epoch 80/100 Loss 1.72187
Epoch 90/100 Loss 1.65757
Epoch 100/100 Loss 1.59909


In [ ]:
from sklearn.metrics import accuracy_score

def evaluate(x):
    model.eval()
    y_pred = model(x)
    y_probas = softmax(y_pred)
    return torch.argmax(y_probas, axis=1)

y_pred = evaluate(torch.from_numpy(X_test).float().cuda())
accuracy_score(y_test, y_pred.cpu().numpy())

0.6918461230748717